### importing libraries

In [1]:
# Copyright 2018 Xanadu Quantum Technologies Inc.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#     http://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Fraud detection fitting script"""
import numpy as np
import os
import tensorflow as tf
import strawberryfields as sf
from strawberryfields.ops import Dgate, BSgate, Kgate, Sgate, Rgate

/home/fraz/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/fraz/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/fraz/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/fraz/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:461: FutureWa

### hyperparameters

In [2]:
# ===================================================================================
#                                   Hyperparameters
# ===================================================================================

# Two modes required: one for "genuine" transactions and one for "fradulent"
mode_number = 2
# Number of photonic quantum layers
depth = 5

# Fock basis truncation
cutoff = 10
# Number of batches in optimization
reps = 1000

# Label for simulation
simulation_label = 1

# Random initialization of gate parameters
sdev_photon = 0.2
sdev = 0.8

# Variable clipping values
disp_clip = 5
sq_clip = 5
kerr_clip = 1

# If loading from checkpoint, previous batch number reached
ckpt_val = 0

# Number of repetitions between each output to TensorBoard
tb_reps = 100
# Number of repetitions between each model save
savr_reps = 100

model_string = str(simulation_label)

# Target location of output
folder_locator = './outputs/'

# Locations of TensorBoard and model save outputs
board_string = folder_locator + 'tensorboard/11/'
checkpoint_string = folder_locator + 'models/11/'

### Loading the training data

In [3]:
# ===================================================================================
#                                   Loading the training data
# ===================================================================================

# Data outputted from data_processor.py
data = np.loadtxt('unsw_train.csv' , delimiter=',')
data_points = len(data)

In [4]:
data_points

82300

In [5]:
# Number of batches to use in the optimization
batch_size = 25

In [6]:
data.shape

(82300, 43)

In [7]:
labels = data[: , 42:]

### Setting up the classical NN input

In [8]:
# ===================================================================================
#                                   Setting up the classical NN input
# ===================================================================================

# Input neurons
input_neurons = 42
# Widths of hidden layers
nn_architecture = [55, 55]
# Output neurons of classical part
output_neurons = 14

# Defining classical network parameters
input_classical_layer = tf.placeholder(tf.float32, shape=[batch_size, input_neurons])

layer_matrix_1 = tf.Variable(tf.random_normal(shape=[input_neurons, nn_architecture[0]]))
offset_1 = tf.Variable(tf.random_normal(shape=[nn_architecture[0]]))

layer_matrix_2 = tf.Variable(tf.random_normal(shape=[nn_architecture[0], nn_architecture[1]]))
offset_2 = tf.Variable(tf.random_normal(shape=[nn_architecture[1]]))

layer_matrix_3 = tf.Variable(tf.random_normal(shape=[nn_architecture[1], output_neurons]))
offset_3 = tf.Variable(tf.random_normal(shape=[output_neurons]))

# Creating hidden layers and output
layer_1 = tf.nn.elu(tf.matmul(input_classical_layer, layer_matrix_1) + offset_1)
layer_2 = tf.nn.elu(tf.matmul(layer_1, layer_matrix_2) + offset_2)

output_layer = tf.nn.elu(tf.matmul(layer_2, layer_matrix_3) + offset_3)

### Defining QNN parameters

In [9]:
# ===================================================================================
#                                   Defining QNN parameters
# ===================================================================================

# Number of beamsplitters in interferometer
bs_in_interferometer = int(1.0 * mode_number * (mode_number - 1) / 2)

with tf.name_scope('variables'):
    bs_variables = tf.Variable(tf.random_normal(shape=[depth, bs_in_interferometer, 2, 2]
                                                , stddev=sdev))
    phase_variables = tf.Variable(tf.random_normal(shape=[depth, mode_number, 2], stddev=sdev))

    sq_magnitude_variables = tf.Variable(tf.random_normal(shape=[depth, mode_number]
                                                          , stddev=sdev_photon))
    sq_phase_variables = tf.Variable(tf.random_normal(shape=[depth, mode_number]
                                                      , stddev=sdev))
    disp_magnitude_variables = tf.Variable(tf.random_normal(shape=[depth, mode_number]
                                                            , stddev=sdev_photon))
    disp_phase_variables = tf.Variable(tf.random_normal(shape=[depth, mode_number]
                                                        , stddev=sdev))
    kerr_variables = tf.Variable(tf.random_normal(shape=[depth, mode_number], stddev=sdev_photon))

parameters = [layer_matrix_1, offset_1, layer_matrix_2, offset_2, layer_matrix_3, offset_3, bs_variables,
              phase_variables, sq_magnitude_variables, sq_phase_variables, disp_magnitude_variables,
              disp_phase_variables, kerr_variables]


### Constructing quantum layers

In [10]:
# ===================================================================================
#                                   Constructing quantum layers
# ===================================================================================


# Defining input QNN layer, whose parameters are set by the outputs of the classical network
def input_qnn_layer():
    with tf.name_scope('inputlayer'):
        Sgate(tf.clip_by_value(output_layer[:, 0], -sq_clip, sq_clip), output_layer[:, 1]) | q[0]
        Sgate(tf.clip_by_value(output_layer[:, 2], -sq_clip, sq_clip), output_layer[:, 3]) | q[1]

        BSgate(output_layer[:, 4], output_layer[:, 5]) | (q[0], q[1])

        Rgate(output_layer[:, 6]) | q[0]
        Rgate(output_layer[:, 7]) | q[1]

        Dgate(tf.clip_by_value(output_layer[:, 8], -disp_clip, disp_clip), output_layer[:, 9]) \
        | q[0]
        Dgate(tf.clip_by_value(output_layer[:, 10], -disp_clip, disp_clip), output_layer[:, 11]) \
        | q[1]

        Kgate(tf.clip_by_value(output_layer[:, 12], -kerr_clip, kerr_clip)) | q[0]
        Kgate(tf.clip_by_value(output_layer[:, 13], -kerr_clip, kerr_clip)) | q[1]


# Defining standard QNN layers
def qnn_layer(layer_number):
    with tf.name_scope('layer_{}'.format(layer_number)):
        BSgate(bs_variables[layer_number, 0, 0, 0], bs_variables[layer_number, 0, 0, 1]) \
        | (q[0], q[1])

        for i in range(mode_number):
            Rgate(phase_variables[layer_number, i, 0]) | q[i]

        for i in range(mode_number):
            Sgate(tf.clip_by_value(sq_magnitude_variables[layer_number, i], -sq_clip, sq_clip),
                  sq_phase_variables[layer_number, i]) | q[i]

        BSgate(bs_variables[layer_number, 0, 1, 0], bs_variables[layer_number, 0, 1, 1]) \
        | (q[0], q[1])

        for i in range(mode_number):
            Rgate(phase_variables[layer_number, i, 1]) | q[i]

        for i in range(mode_number):
            Dgate(tf.clip_by_value(disp_magnitude_variables[layer_number, i], -disp_clip,
                                   disp_clip), disp_phase_variables[layer_number, i]) | q[i]

        for i in range(mode_number):
            Kgate(tf.clip_by_value(kerr_variables[layer_number, i], -kerr_clip, kerr_clip)) | q[i]

### Defining QNN

In [11]:
# ===================================================================================
#                                   Defining QNN
# ===================================================================================

# construct the two-mode Strawberry Fields program
prog = sf.Program(mode_number)

# construct the circuit
with prog.context as q:
    input_qnn_layer()

    for i in range(depth):
        qnn_layer(i)

# create an engine
eng = sf.Engine('tf', backend_options={"cutoff_dim": cutoff, "batch_size": batch_size})

# run the engine (in batch mode)
state = eng.run(prog, run_options={"eval": False}).state
# extract the state
ket = state.ket()

### Setting up cost function

In [12]:
# ===================================================================================
#                                   Setting up cost function
# ===================================================================================

# Classifications for whole batch: rows act as data points in the batch and columns
# are the one-hot classifications

func_to_minimise = 0
classification = tf.placeholder(shape=[batch_size, 2], dtype=tf.int32)

# Building up the function to minimize by looping through batch
for i in range(batch_size):
    # Probabilities corresponding to a single photon in either mode
    prob = tf.abs(ket[i, classification[i, 0], classification[i, 1]]) ** 2
    # These probabilities should be optimised to 1
    func_to_minimise += (1.0 / batch_size) * (prob-1) ** 2

# Defining the cost function
cost_func = func_to_minimise
tf.summary.scalar('Cost', cost_func)

<tf.Tensor 'Cost:0' shape=() dtype=string>

### Training

In [13]:
import time
start_time = time.time()
print(time.asctime( time.localtime(time.time()) ))

Sat Nov 30 11:50:14 2019


In [14]:
# ===================================================================================
#                                   Training
# ===================================================================================

# We choose the Adam optimizer
optimiser = tf.train.AdamOptimizer(learning_rate=0.001 , beta1=0.05)
training = optimiser.minimize(cost_func)

# Saver/Loader for outputting model
saver = tf.train.Saver(parameters)
data
session = tf.Session()
session.run(tf.global_variables_initializer())

# Load previous model if non-zero ckpt_val is specified
if ckpt_val != 0:
    saver.restore(session, checkpoint_string + 'sess.ckpt-' + str(ckpt_val))

# TensorBoard writer
writer = tf.summary.FileWriter(board_string)
merge = tf.summary.merge_all()

counter = ckpt_val

# Tracks optimum value found (set high so first iteration encodes value)
opt_val = 1e20
# Batch number in which optimum value occurs
opt_position = 0
# Flag to detect if new optimum occured in last batch
new_opt = False

#TIME
import time
start_time = time.time()
print(time.asctime( time.localtime(time.time()) ))

while counter <= reps:
    # Shuffles data to create new epoch
    np.random.shuffle(data)

    # Splits data into batches
    split_data = np.split(data, data_points / batch_size)

    for batch in split_data:
        print("++++++++++++++")
        if counter > reps:
            break

        # Input data (provided as principal components)
        #data_points_principal_components = batch[:, 1:input_neurons + 1]
        classes = batch[: , 42:]
        batch_data = batch[: , :42]
        # Data classes

        #classes = batch[: ,42: ]
        #print(data_points_principal_components.shape)
        # Encoding classes into one-hot form
        one_hot_input = np.zeros((batch_size, 2))
        
        for i in range(batch_size):
            if classes[i] == 0:
                # Encoded such that genuine transactions should be outputted as a photon in the first mode
                one_hot_input[i] = [1, 0]
            else:
                one_hot_input[i] = [0, 1]

        
        #print("one hot shape : ",one_hot_input.shape)
        #print("data shape : " ,batch_data.shape)
        
        # Output to TensorBoard
        if counter % tb_reps == 0:
            [summary, training_run, func_to_minimise_run] = session.run([merge, training, func_to_minimise],
                                                                        feed_dict={
                                                                            input_classical_layer:
                                                                                batch_data,
                                                                            classification: one_hot_input
                                                                            })
            writer.add_summary(summary, counter)
            print("summary :  ",summary)

        else:
            # Standard run of training
            [training_run, func_to_minimise_run] = session.run([training, func_to_minimise], feed_dict={
                input_classical_layer: batch_data, classification: one_hot_input})
            
        # Ensures cost function is well behaved
        if np.isnan(func_to_minimise_run):
            compute_grads = session.run(optimiser.compute_gradients(cost_func),
                                        feed_dict={input_classical_layer: batch_data,
                                                   classification: one_hot_input})
            print("compute grads:  ",compute_grads)

            if not os.path.exists(checkpoint_string):
                os.makedirs(checkpoint_string)
            # If cost function becomes NaN, output value of gradients for investigation
            np.save(checkpoint_string + 'NaN.npy', compute_grads)
            print('NaNs outputted - leaving at step ' + str(counter))
            raise SystemExit

        # Test to see if new optimum found in current batch
        if func_to_minimise_run < opt_val:
            opt_val = func_to_minimise_run
            opt_position = counter
            new_opt = True

        # Save model every fixed number of batches, provided a new optimum value has occurred
        if (counter % savr_reps == 0) and (i != 0) and new_opt and (not np.isnan(func_to_minimise_run)):
            if not os.path.exists(checkpoint_string):
                os.makedirs(checkpoint_string)
            saver.save(session, checkpoint_string + 'sess.ckpt', global_step=counter)
            # Saves position of optimum and corresponding value of cost function
            np.savetxt(checkpoint_string + 'optimum.txt', [opt_position, opt_val])

        counter += 1
        print("iteration: ",counter , " cost: ",func_to_minimise_run)
        print("training_run  :   ",training_run)

end_time = time.time() - start_time
print("--- %s seconds ---" % (time.time() - start_time))
print (time.asctime( time.localtime(time.time()) ))

Sat Nov 30 11:50:37 2019
++++++++++++++
summary :   b'\n\x0b\n\x04Cost\x15KW|?'
iteration:  1  cost:  0.985707
training_run  :    None
++++++++++++++
iteration:  2  cost:  0.9935115
training_run  :    None
++++++++++++++
iteration:  3  cost:  0.9852517
training_run  :    None
++++++++++++++
iteration:  4  cost:  0.9743221
training_run  :    None
++++++++++++++
iteration:  5  cost:  0.9988628
training_run  :    None
++++++++++++++
iteration:  6  cost:  0.99048823
training_run  :    None
++++++++++++++
iteration:  7  cost:  0.99581426
training_run  :    None
++++++++++++++
iteration:  8  cost:  0.99553406
training_run  :    None
++++++++++++++
iteration:  9  cost:  0.99431634
training_run  :    None
++++++++++++++
iteration:  10  cost:  0.9999333
training_run  :    None
++++++++++++++
iteration:  11  cost:  0.9957647
training_run  :    None
++++++++++++++
iteration:  12  cost:  0.99277896
training_run  :    None
++++++++++++++
iteration:  13  cost:  0.98489183
training_run  :    None
+++

iteration:  113  cost:  0.98259646
training_run  :    None
++++++++++++++
iteration:  114  cost:  0.9935869
training_run  :    None
++++++++++++++
iteration:  115  cost:  0.9780497
training_run  :    None
++++++++++++++
iteration:  116  cost:  0.9753325
training_run  :    None
++++++++++++++
iteration:  117  cost:  0.9654169
training_run  :    None
++++++++++++++
iteration:  118  cost:  0.9914068
training_run  :    None
++++++++++++++
iteration:  119  cost:  0.9778025
training_run  :    None
++++++++++++++
iteration:  120  cost:  0.97948945
training_run  :    None
++++++++++++++
iteration:  121  cost:  0.9570055
training_run  :    None
++++++++++++++
iteration:  122  cost:  0.94329613
training_run  :    None
++++++++++++++
iteration:  123  cost:  0.9572291
training_run  :    None
++++++++++++++
iteration:  124  cost:  0.9639725
training_run  :    None
++++++++++++++
iteration:  125  cost:  0.9978491
training_run  :    None
++++++++++++++
iteration:  126  cost:  0.99112606
training_run 

iteration:  225  cost:  0.981477
training_run  :    None
++++++++++++++
iteration:  226  cost:  0.94889206
training_run  :    None
++++++++++++++
iteration:  227  cost:  0.9876252
training_run  :    None
++++++++++++++
iteration:  228  cost:  0.9492252
training_run  :    None
++++++++++++++
iteration:  229  cost:  0.9743072
training_run  :    None
++++++++++++++
iteration:  230  cost:  0.98923063
training_run  :    None
++++++++++++++
iteration:  231  cost:  0.9933335
training_run  :    None
++++++++++++++
iteration:  232  cost:  0.98746777
training_run  :    None
++++++++++++++
iteration:  233  cost:  0.9746673
training_run  :    None
++++++++++++++
iteration:  234  cost:  0.97960687
training_run  :    None
++++++++++++++
iteration:  235  cost:  0.93776035
training_run  :    None
++++++++++++++
iteration:  236  cost:  0.9887092
training_run  :    None
++++++++++++++
iteration:  237  cost:  0.962954
training_run  :    None
++++++++++++++
iteration:  238  cost:  0.97736657
training_run 

iteration:  337  cost:  0.9700589
training_run  :    None
++++++++++++++
iteration:  338  cost:  0.9771159
training_run  :    None
++++++++++++++
iteration:  339  cost:  0.9863078
training_run  :    None
++++++++++++++
iteration:  340  cost:  0.97991884
training_run  :    None
++++++++++++++
iteration:  341  cost:  0.96578074
training_run  :    None
++++++++++++++
iteration:  342  cost:  0.98237157
training_run  :    None
++++++++++++++
iteration:  343  cost:  0.987124
training_run  :    None
++++++++++++++
iteration:  344  cost:  0.9978811
training_run  :    None
++++++++++++++
iteration:  345  cost:  0.9467792
training_run  :    None
++++++++++++++
iteration:  346  cost:  0.95082456
training_run  :    None
++++++++++++++
iteration:  347  cost:  0.9824507
training_run  :    None
++++++++++++++
iteration:  348  cost:  0.99580723
training_run  :    None
++++++++++++++
iteration:  349  cost:  0.96516234
training_run  :    None
++++++++++++++
iteration:  350  cost:  0.9939929
training_run

iteration:  449  cost:  0.96235645
training_run  :    None
++++++++++++++
iteration:  450  cost:  0.9560499
training_run  :    None
++++++++++++++
iteration:  451  cost:  0.9548337
training_run  :    None
++++++++++++++
iteration:  452  cost:  0.9691348
training_run  :    None
++++++++++++++
iteration:  453  cost:  0.9690912
training_run  :    None
++++++++++++++
iteration:  454  cost:  0.96104926
training_run  :    None
++++++++++++++
iteration:  455  cost:  0.9420056
training_run  :    None
++++++++++++++
iteration:  456  cost:  0.9821694
training_run  :    None
++++++++++++++
iteration:  457  cost:  0.9797713
training_run  :    None
++++++++++++++
iteration:  458  cost:  0.9882207
training_run  :    None
++++++++++++++
iteration:  459  cost:  0.9815921
training_run  :    None
++++++++++++++
iteration:  460  cost:  0.96029234
training_run  :    None
++++++++++++++
iteration:  461  cost:  0.9684313
training_run  :    None
++++++++++++++
iteration:  462  cost:  0.97445685
training_run 

iteration:  561  cost:  0.9677032
training_run  :    None
++++++++++++++
iteration:  562  cost:  0.9466276
training_run  :    None
++++++++++++++
iteration:  563  cost:  0.95108783
training_run  :    None
++++++++++++++
iteration:  564  cost:  0.9810628
training_run  :    None
++++++++++++++
iteration:  565  cost:  0.98283505
training_run  :    None
++++++++++++++
iteration:  566  cost:  0.94572026
training_run  :    None
++++++++++++++
iteration:  567  cost:  0.93718153
training_run  :    None
++++++++++++++
iteration:  568  cost:  0.96891916
training_run  :    None
++++++++++++++
iteration:  569  cost:  0.97598636
training_run  :    None
++++++++++++++
iteration:  570  cost:  0.9518979
training_run  :    None
++++++++++++++
iteration:  571  cost:  0.90964484
training_run  :    None
++++++++++++++
iteration:  572  cost:  0.97357845
training_run  :    None
++++++++++++++
iteration:  573  cost:  0.9870568
training_run  :    None
++++++++++++++
iteration:  574  cost:  0.94160295
training

iteration:  673  cost:  0.9349328
training_run  :    None
++++++++++++++
iteration:  674  cost:  0.9470625
training_run  :    None
++++++++++++++
iteration:  675  cost:  0.96159405
training_run  :    None
++++++++++++++
iteration:  676  cost:  0.939526
training_run  :    None
++++++++++++++
iteration:  677  cost:  0.9199638
training_run  :    None
++++++++++++++
iteration:  678  cost:  0.95108545
training_run  :    None
++++++++++++++
iteration:  679  cost:  0.9411211
training_run  :    None
++++++++++++++
iteration:  680  cost:  0.9251103
training_run  :    None
++++++++++++++
iteration:  681  cost:  0.9660619
training_run  :    None
++++++++++++++
iteration:  682  cost:  0.8962008
training_run  :    None
++++++++++++++
iteration:  683  cost:  0.9264342
training_run  :    None
++++++++++++++
iteration:  684  cost:  0.9168843
training_run  :    None
++++++++++++++
iteration:  685  cost:  0.94013757
training_run  :    None
++++++++++++++
iteration:  686  cost:  0.94926614
training_run  

iteration:  784  cost:  0.8904037
training_run  :    None
++++++++++++++
iteration:  785  cost:  0.9047171
training_run  :    None
++++++++++++++
iteration:  786  cost:  0.9229209
training_run  :    None
++++++++++++++
iteration:  787  cost:  0.934264
training_run  :    None
++++++++++++++
iteration:  788  cost:  0.89697695
training_run  :    None
++++++++++++++
iteration:  789  cost:  0.9884876
training_run  :    None
++++++++++++++
iteration:  790  cost:  0.9346393
training_run  :    None
++++++++++++++
iteration:  791  cost:  0.9680669
training_run  :    None
++++++++++++++
iteration:  792  cost:  0.9313702
training_run  :    None
++++++++++++++
iteration:  793  cost:  0.91497296
training_run  :    None
++++++++++++++
iteration:  794  cost:  0.9411014
training_run  :    None
++++++++++++++
iteration:  795  cost:  0.94771284
training_run  :    None
++++++++++++++
iteration:  796  cost:  0.9278572
training_run  :    None
++++++++++++++
iteration:  797  cost:  0.94288456
training_run  

iteration:  896  cost:  0.89461243
training_run  :    None
++++++++++++++
iteration:  897  cost:  0.9317103
training_run  :    None
++++++++++++++
iteration:  898  cost:  0.90837365
training_run  :    None
++++++++++++++
iteration:  899  cost:  0.92874825
training_run  :    None
++++++++++++++
iteration:  900  cost:  0.92129695
training_run  :    None
++++++++++++++
summary :   b'\n\x0b\n\x04Cost\x15\xd2\xddf?'
iteration:  901  cost:  0.9018222
training_run  :    None
++++++++++++++
iteration:  902  cost:  0.9427431
training_run  :    None
++++++++++++++
iteration:  903  cost:  0.9276906
training_run  :    None
++++++++++++++
iteration:  904  cost:  0.9004612
training_run  :    None
++++++++++++++
iteration:  905  cost:  0.9672557
training_run  :    None
++++++++++++++
iteration:  906  cost:  0.8851419
training_run  :    None
++++++++++++++
iteration:  907  cost:  0.89312655
training_run  :    None
++++++++++++++
iteration:  908  cost:  0.9154116
training_run  :    None
++++++++++++++
